In [1]:
from os import listdir
import jsonlines
import itertools
import nltk
from nltk.corpus import stopwords
import pymorphy2 as pm
import networkx as nx
import re
from bisect import bisect_left
#from joblib import Parallel, delayed
from collections import Counter
from tqdm import trange
from tqdm import tqdm
from math import log
from random import shuffle
from sys import exit
import numpy as np
import json
import codecs
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.neighbors import NearestCentroid
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from scipy.sparse import load_npz
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack

from sklearn.externals import joblib

In [15]:
ok_set = set()
with open("sources/accepted_categories.txt", mode="r", encoding="utf-8") as inp:
    for line in inp:
        line = line[:-1]
        ok_set.add(line)

ok = sorted(ok_set)

categories_dict = {}
with open("sources/article_cat.json", mode="r") as input:
    categories_dict = json.loads(input.read())

#X = load_npz("sources/tf_idf.npz")
#class_centroids = load_npz("sources/centroids_titles_bigrams.npz")
#X_titles = load_npz("sources/X_titles_bigrams.npz")

X = np.load("sources/svd500.npy").astype(np.float16)
class_centroids = np.load("sources/centroids_svd500.npy").astype(np.float16)
X_titles = X
    
ids = sorted([item['id'] for item in jsonlines.open('sources/normalized_texts.jl', 'r')])

In [16]:
class_centroids.shape

(14697, 500)

In [17]:
X.shape

(96794, 500)

In [18]:
k_nearest = 5

kn_clust = NearestNeighbors(n_neighbors=k_nearest)
kn_clust.fit(class_centroids)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [6]:
#for sparce features
X_left = []
for x in tqdm(X):
    for j in range(k_nearest):
        X_left.append(x)       
X_left = vstack(X_left)

X_right = []
y = []
rows = []
batchsize = 1000
for i in trange(0, X.shape[0], batchsize):
    curr_x = X[i:i+batchsize]
    _neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
    
    for j, neighbors in enumerate(_neighbors):
        cats = categories_dict[ids[i+j]]
        neighbors = [ok[c] for c in neighbors]
        common = set(neighbors).intersection(set(cats))
        for neigh in neighbors:
            rows.append(ok.index(neigh))
            y.append(1 if neigh in common else 0)

X_right = class_centroids[rows]
X_full = hstack([X_left, X_right])
y = np.array(y)

96794it [00:04, 23128.30it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [08:14<00:00,  5.10s/it]


In [19]:
#for dence features
X_left = []
for x in tqdm(X):
    for j in range(k_nearest):
        X_left.append(x)       
X_left = np.array(X_left)

X_right = []
y = []
rows = []
batchsize = 1000
for i in trange(0, X.shape[0], batchsize):
    curr_x = X[i:i+batchsize]
    _neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
    
    for j, neighbors in enumerate(_neighbors):
        cats = categories_dict[ids[i+j]]
        neighbors = [ok[c] for c in neighbors]
        common = set(neighbors).intersection(set(cats))
        for neigh in neighbors:
            rows.append(ok.index(neigh))
            y.append(1 if neigh in common else 0)

X_right = class_centroids[rows]
X_full = np.concatenate((X_left, X_right), axis=1)
y = np.array(y)

100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [24:21<00:00, 15.07s/it]


In [7]:
#when X and class_centroids have different shape[1] (tf-idf + titles)
X_left = []
for x in tqdm(X):
    for j in range(k_nearest):
        X_left.append(x)       
X_left = vstack(X_left)

X_right = []
y = []
rows = []
batchsize = 1000
for i in trange(0, X.shape[0], batchsize):
    curr_x = X_titles[i:i+batchsize]
    _neighbors = kn_clust.kneighbors(curr_x, n_neighbors=k_nearest, return_distance=False)
    
    for j, neighbors in enumerate(_neighbors):
        cats = categories_dict[ids[i+j]]
        neighbors = [ok[c] for c in neighbors]
        common = set(neighbors).intersection(set(cats))
        for neigh in neighbors:
            rows.append(ok.index(neigh))
            y.append(1 if neigh in common else 0)

X_right = class_centroids[rows]
X_full = hstack([X_left, X_right])
y = np.array(y)

96794it [00:04, 21649.62it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 97/97 [01:39<00:00,  1.02s/it]


In [20]:
print(sum(y), len(y))

118137 483970


In [ ]:
for c in [0.1, 1.0, 5, 10, 20]:
    clf = LogisticRegression(C=c, class_weight="balanced")
    scoring = {'accu': 'accuracy'}
    scores = cross_validate(clf, X_full, y, cv=3, return_train_score=True, scoring=scoring)
    print("C = " + str(c))
    print(scores)

C:\Users\Maxim\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\Maxim\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


C = 0.1
{'fit_time': array([91.27764368, 94.64258122, 78.35975432]), 'score_time': array([1.8619504 , 1.76569438, 1.70653677]), 'test_accu': array([0.5699214 , 0.549872  , 0.69366426]), 'train_accu': array([0.6377206 , 0.65372993, 0.61591461])}


C:\Users\Maxim\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\Maxim\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


C = 1.0
{'fit_time': array([231.68989396, 205.8109622 , 163.73263597]), 'score_time': array([1.92160892, 1.72458482, 2.08755064]), 'test_accu': array([0.56837172, 0.55315113, 0.68884784]), 'train_accu': array([0.64902711, 0.66385554, 0.61458808])}


C:\Users\Maxim\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\Maxim\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


C = 5
{'fit_time': array([375.29845953, 371.31261945, 342.52797055]), 'score_time': array([1.81383038, 1.77070642, 1.80981112]), 'test_accu': array([0.56768367, 0.55266763, 0.68776306]), 'train_accu': array([0.65136093, 0.66589802, 0.61371406])}


C:\Users\Maxim\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)
C:\Users\Maxim\Anaconda3\lib\site-packages\numpy\core\_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


C = 10
{'fit_time': array([436.94964576, 428.39220166, 425.00939941]), 'score_time': array([1.72659039, 1.8078053 , 1.69751263]), 'test_accu': array([0.56741092, 0.55258085, 0.68791803]), 'train_accu': array([0.65155929, 0.6660251 , 0.61310348])}


In [10]:
i = 0
n_splits = 3
kf = KFold(n_splits=n_splits, shuffle=True, random_state=27)
X_full = X_full.tocsr()
for item in kf.split(ids):
    train_index, test_index = item[0], item[1]
    
    print("TRAIN:", train_index, "TEST:", test_index)
    print(len(train_index), len(test_index))
    clf = LogisticRegression(C=10, class_weight="balanced")
    clf.fit(X_full[train_index], y[train_index])
    joblib.dump(clf, "sources/binlogreg_clfs/binlogreg_tfidf_titlesbigrams" + str(i) +".pkl")
    i += 1

TRAIN: [    0     1     2 ... 96790 96792 96793] TEST: [    3     4     7 ... 96785 96789 96791]
64529 32265
TRAIN: [    2     3     4 ... 96791 96792 96793] TEST: [    0     1     5 ... 96787 96788 96790]
64529 32265
TRAIN: [    0     1     3 ... 96789 96790 96791] TEST: [    2     6     9 ... 96786 96792 96793]
64530 32264


In [9]:
X_full.shape

(483970, 130597)